## CosmosDB/Mongo API Sales Data Processing with Synapse Link

The purpose of this Spark Notebook is to read the **Synapse Link Sales data** 
and produce **aggregated totals by Customer** and write these aggregated totals 
to both **Azure Storage** and **Azure PostgreSQL**.

### Programming Logic in this Notebook, by Cell

- Define common PySpark functions
- Load the Synapse Link Sales data into a Dataframe
- Sort by _ts, display the newest Documents
- Select just the pertinent columns for the aggregation calculation
- Aggregate Sales by Customer
- Pass the aggregated DataFrame to Spark/Scala as a TempView
- Write the aggregated DataFrame to Azure Storage as CSV 
- Write the aggregated DataFrame to an Azure PostgreSQL table

### Links 

- https://github.com/Azure-Samples/Synapse/tree/main/Notebooks/PySpark

In [ ]:
# Define common PySpark functions used in this Notebook

def print_df_shape(df, msg):
    print("shape of df {} - row count: {}, column count: {}".format(
        msg, str(df.count()), str(len(df.columns)) ))

def write_df_to_csv_blob(df, out_csv):

    # Azure storage account info
    blob_account_name   = 'cjoakimstorage'
    blob_container_name = 'retail'
    blob_relative_path  = 'demo'
    linked_service_name = 'SecondaryAzureBlobStorage'

    blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(
        linked_service_name)
    #print('blob_sas_token: {}'.format(blob_sas_token))

    wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (
        blob_container_name, blob_account_name, blob_relative_path)

    spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (
        blob_container_name, blob_account_name), blob_sas_token)

    csv_path = '{}/{}'.format(wasbs_path, out_csv)

    print('wasbs_path: ' + wasbs_path)
    print('csv_path:   ' + csv_path)

    # Write to blob storage, coalesce it into one CSV file
    df.coalesce(1).write.csv(csv_path, mode='overwrite', header='true')
    print('written')


In [ ]:
# Load the SynapseLink Sales data into a Dataframe.
# Select just the "sale" document types from the sales container, 
# which have a minimum _ts (timestamp) value.

# The documents in CosmosDB OLTP data look like this:
# { 
#     "_id" : ObjectId("6200059edbf78e1f05346e70"), 
#     "pk" : "1", 
#     "id" : "d6167c84-024a-4ecd-9c95-55f0005615d0", 
#     "sale_id" : NumberInt(1), 
#     "doctype" : "sale", 
#     "date" : "2021-01-01", 
#     "dow" : "fri", 
#     "customer_id" : NumberInt(3275), 
#     "store_id" : NumberInt(61), 
#     "item_count" : NumberInt(3), 
#     "total_cost" : 2049.71
# }

# The above document is from this query in Studio 3T:
#   db.getCollection("sales").find({doctype:"sale"})

from pyspark.sql.functions import col
import pyspark.sql.functions as F 

# initialize variables
min_doc_timestamp = 1640995200  # 2022-01-01T00:00:00.000Z
max_doc_timestamp = 1672531199  # 2022-12-31T23:59:59.999Z

# read the raw OLAP data, filtering by document _ts (timestamp)
df_sales_raw = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "CosmosMongoDemoDB")\
    .option("spark.cosmos.container", "sales")\
    .load()\
    .filter(col("_ts") >= min_doc_timestamp)\
    .filter(col("_ts") <= max_doc_timestamp)

print_df_shape(df_sales_raw, "df_sales_raw")
display(df_sales_raw.limit(20))
df_sales_raw.printSchema()

# shape of df df_sales_raw - row count: 110025, column count: 20

In [ ]:
# Sort by _ts, display the newest Documents

from pyspark.sql.functions import col
import pyspark.sql.functions as F 

sorted_df = df_sales_raw.orderBy(col("_ts").desc())

print_df_shape(sorted_df, "sorted_df")

sorted_df.select("sale_id", "doctype", "date", "_ts").show(30)

# |sale_id|    doctype|        date|       _ts|
# |{31396}|{line_item}|{2022-02-16}|1644173836|


In [ ]:
# Select just the pertinent columns for the aggregation calculation.
# Unpack the struct columns with attrname.* syntax.

df_sales_unpacked = df_sales_raw.select(
    col('doctype.*'),
    col('date.*'),
    col('customer_id.*'),
    col('item_count.*'),
    col('total_cost.*'))

print_df_shape(df_sales_unpacked, "df_sales_unpacked")
display(df_sales_unpacked.limit(3))
df_sales_unpacked.printSchema()

# Rename the columns of the unpacked DataFrame
new_column_names = [
    'doctype', 'date', 'customer_id', 'item_count', 'total_cost']
df_sales = df_sales_unpacked.toDF(*new_column_names).filter(col("doctype") == "sale")

print_df_shape(df_sales, "df_sales")
display(df_sales.limit(3))
df_sales.printSchema()


In [ ]:
# Aggregate Sales by Customer 

# Consider an alternative implementation using just CosmosDB, and not Spark:
# 1. Read customers container to get the unique set of customer ids (cross partition)
# 2. Loop through the customer id list:
#    - Read all of the sale documents for each customer in the timeframe (cross partition)
#    - sum the sales item_count and item_count for each customer (memory intensive?)

import pyspark.sql.functions as F 

df_customer_aggregated = df_sales.groupBy("customer_id") \
    .agg(
        F.first('customer_id').alias('_id'), \
        F.first('customer_id').alias('pk'), \
        F.count("customer_id").alias('order_count'), \
        F.sum("total_cost").alias("total_dollar_amount"), \
        F.sum("item_count").alias("total_item_count")) \
        .sort("customer_id", ascending=True)

print_df_shape(df_customer_aggregated, "df_customer_aggregated")
display(df_customer_aggregated.limit(4))
df_customer_aggregated.printSchema()

# Pass the df_customer_aggregated DataFrame to Spark/Scala as a TempView
df_customer_aggregated.createOrReplaceTempView("CustomerAggSales")

print('df_customer_aggregated, shape: {} x {}'.format(
        df_customer_aggregated.count(), 
        len(df_customer_aggregated.columns)))


In [ ]:
# Write the df_customer_aggregated to Azure Blob Storage as CSV 

from pyspark.sql import SparkSession
from pyspark.sql.types import *

import pyspark.sql.functions as F 

# Azure storage account info
blob_account_name   = 'cjoakimstorage'
blob_container_name = 'synapse'
blob_relative_path  = 'retail/sales/mongo/'
linked_service_name = 'SecondaryAzureBlobStorage'

blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name)
print('blob_sas_token: {}'.format(blob_sas_token))

# Allow Spark to access from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (
    blob_container_name, blob_account_name, blob_relative_path)

spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (
    blob_container_name, blob_account_name), blob_sas_token)

csv_path = '{}{}'.format(wasbs_path,'sales_by_customer_csv')

print('wasbs_path: ' + wasbs_path)
print('csv_path:   ' + csv_path)

# Write to blob storage, coalesce it into one CSV file
df_customer_aggregated.coalesce(1).write.csv(csv_path, mode='overwrite', header='true')
print('csv data written to azure storage blob')


In [ ]:
%%spark 

// Obtain configuration values from spark.conf
val connStr  = spark.conf.get("spark.azurepg.jdbc.connstring")
val driver   = spark.conf.get("spark.azurepg.jdbc.driver")
val server   = spark.conf.get("spark.azurepg.jdbc.server")
val database = spark.conf.get("spark.azurepg.jdbc.database")
val table    = "public.customer_sales"
val user     = spark.conf.get("spark.azurepg.jdbc.user")
val password = spark.conf.get("spark.azurepg.jdbc.pass")

// Read the temp table into a Dataframe
val df_temp_view = spark.read.table("CustomerAggSales")

println("df_temp_view row count: " + df_temp_view.count())

// Using JDBC, write the Dataframe to Azure PostgreSQL
df_temp_view.write
  .format("jdbc")
  .option("url", connStr)
  .option("driver", driver)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .mode("overwrite")
  .save()

// In Azure Data Studio, execute this SQL query vs PostgreSQL:
// select * from public.customer_sales order by customer_id 

println("done\n\n\n\n")
